In [ ]:
#---------------------------------------------------------------------
#This code finds the Trump style words
#---------------------------------------------------------------------

import nltk

#---------------------------------------------------------------------
#General word rank
#---------------------------------------------------------------------

#from: http://www.anc.org/data/anc-second-release/frequency-data/

#create a dictionary with the most frequent generally used words and their respective rank
word_rank_file = open("..\\data\\wordranking.txt")
word_rank_text = word_rank_file.read()
word_freq_dict = {}
for word_rank in word_rank_text.split("\n"):
    word = word_rank.split()[1].lower()
    if word in word_freq_dict: #words are counted separately for each POS tag, so we sum them up
        word_freq_dict[word] = word_freq_dict[word]+ int(word_rank.split()[3])
    else:
        word_freq_dict[word] = int(word_rank.split()[3])

word_rank_dict={key: rank for rank, key in enumerate(sorted(word_freq_dict, key=word_freq_dict.get, reverse=True), 1)}

#---------------------------------------------------------------------
#Trump word rank
#---------------------------------------------------------------------

#load the cleaned Trump corpus
final_trump_file = open("..\\data\\final_trump_corpus.txt")
final_trump_corpus = final_trump_file.read().split("\n")

#final_trump_corpus=load("final_trump_corpus.txt")
tokenized_text = [nltk.word_tokenize(sentence) for sentence in final_trump_corpus]

#reconnect @ to name -> @barackobama
final_text=[]
for tokenized_sentence in tokenized_text:
    skip =[]
    for idx,word in enumerate(tokenized_sentence):
        if idx not in skip:
            if word=="@":
                final_text.append("@"+tokenized_sentence[idx+1])
                skip.append(idx+1)
            else:
                final_text.append(word.lower())
                
freq_dist = nltk.FreqDist(final_text)
freq_list = freq_dist.most_common()

#create a word rank dict for the Trump corpus
trump_word_rank_dict = {}
for rank, word in enumerate(freq_list[:500]):
    trump_word_rank_dict[word[0]]= rank+1

#---------------------------------------------------------------------
#Compare the word rankings
#---------------------------------------------------------------------

compare_dict={}
not_in_5000 = {} #the words not appearing in the top 5000 at all

#irrelevant words (for example 're with different '’` etc.)
irrelevant = ["'re", "'m", "is", "are", "was", "were","am", "has", "have", "t", "do", "does", "did", "don", "did"]
for word,rank in trump_word_rank_dict.items():
    if word not in '!"#$%&\'\'’()*+,--.../:;?@[\\]^_{|}~``“”1023456789' and word not in irrelevant:
        if word in word_rank_dict:
            if (word_rank_dict[word]-rank)>40:
                compare_dict[word] = word_rank_dict[word]-rank
        else:
            not_in_5000[word] = rank
            
#list of Trumps style words
trump_style_words = [ word for word, score in compare_dict.items()]
trump_style_words.extend([ word for word, score in not_in_5000.items()])

#---------------------------------------------------------------------
#Create the dictionary with Trump style words and occurrences
#---------------------------------------------------------------------

#open and load topic corpus
trump_file_tagged = open("..\\data\\stanford_tagged_trump_corpus.txt", encoding = "utf-8")
trump_corpus_tagged = eval(trump_file_tagged.read())

#dictionary to collect info about the Trump style words
trump_style_dict = {}

for sentence in trump_corpus_tagged:
    for idx, (word, tag) in enumerate(sentence):
        if word.lower() in trump_style_words: 
            
            #find the tag of the word, before the Trump style word
            if idx==0:
                prior_tag="START"
            else:
                prior_tag = sentence[idx-1][1]
                
            #find the tag of the word after the Trump style word    
            try:
                next_tag = sentence[idx+1][1]
            except IndexError:
                next_tag = "END"
            
            #create dictionary with counts of each occurance
            if (prior_tag,tag,next_tag) not in trump_style_dict:
                trump_style_dict[prior_tag,tag,next_tag]={}
            if word.lower() not in trump_style_dict[prior_tag,tag,next_tag]:
                trump_style_dict[prior_tag,tag,next_tag][word.lower()]=1
            else:
                trump_style_dict[prior_tag,tag,next_tag][word.lower()]=trump_style_dict[prior_tag,tag,next_tag][word.lower()]+1

#---------------------------------------------------------------------
#Find the words that occur frequently in a certain context
#---------------------------------------------------------------------

style_dict={}
for entry in trump_style_dict:
    style_dict[entry]={}
    for word,count in trump_style_dict[entry].items():
        if count > 50:
            style_dict[entry][word]=count

final_style_dict={}
for entry in style_dict:
    if style_dict[entry]!={}:
        final_style_dict[entry]=style_dict[entry]